In [1]:
import os
from dotenv import load_dotenv

# --- LangChain & Google Gemini Imports ---
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load environment variables from .env file
load_dotenv()

True

In [2]:
# --- Configuration ---
FAISS_INDEX_PATH = "faiss_store_finfriend_gemini"
DOC_DIRECTORY = "./documents"

# Check if the API key is available
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY not found. Please set it in your .env file.")

In [3]:
# --- Initialize Models ---
print("Initializing Google Gemini models...")

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(
    model_name='all-MiniLM-L6-v2'
)

# Initialize the language model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

print("Models initialized successfully! ✅")

Initializing Google Gemini models...


C:\Users\bhoge\AppData\Local\Temp\ipykernel_5740\2391077354.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Models initialized successfully! ✅


In [4]:
# --- Ingestion Process ---
print(f"Processing documents from '{DOC_DIRECTORY}'...")

# Load all .txt files from the directory
loader = TextLoader("manageMoney.txt",encoding="utf-8")
documents = loader.load()

# Split documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Split documents into {len(chunks)} chunks.")

# Create FAISS index from the chunks
print("Creating and saving FAISS index...")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)

# Save the FAISS index locally
vectorstore.save_local(FAISS_INDEX_PATH)

print(f"Index created and saved to '{FAISS_INDEX_PATH}'! ✅")

Processing documents from './documents'...
Split documents into 112 chunks.
Creating and saving FAISS index...
Index created and saved to 'faiss_store_finfriend_gemini'! ✅


In [5]:
# A concise and direct set of instructions for the model
FIN_FRIEND_PROMPT_INSTRUCTIONS = """
**INSTRUCTIONS:**
You are Fin-Friend, an expert and empathetic financial guide in India. Your goal is to conduct a financial health check-up.
- Analyze the user's data below using the provided context from the financial guide.
- NEVER give direct financial advice. Frame all outputs as "educational information" or "examples."
- Your final analysis MUST end with the disclaimer: "This is for informational purposes only and is not financial advice. Please consult with a qualified financial advisor before making any decisions."
"""

def format_user_data_for_llm(data):
    """
    Formats the collected user data into a clean, human-readable string
    for the language model.
    """
    report_lines = []
    for key, value in data.items():
        title = key.replace('_', ' ').title()
        
        if isinstance(value, dict): # Correctly formats the nested expenses dictionary
            report_lines.append(f"**{title}:**")
            for sub_key, sub_value in value.items():
                sub_title = sub_key.replace('_', ' ').title()
                report_lines.append(f"- {sub_title}: {sub_value}")
        else:
            report_lines.append(f"**{title}:**\n{value}")
        
        report_lines.append("") # Adds a blank line for better readability
        
    return "\n".join(report_lines)

def get_user_input(prompt_text):
    """A helper function to handle user input."""
    return input(prompt_text + "\n> ")

In [ ]:
# --- Phase 1: Structured Data Gathering ---
user_data = {}
expenses = {}

print("Hello! I'm Fin-Friend... Let's get started!\n")

# Gather income
user_data['income_salary'] = get_user_input("First, what is your fixed monthly take-home salary?")
user_data['income_other'] = get_user_input("Do you have any other sources of income (like freelance work or bonuses)?")

# Gather expenses one by one for clean data
print("\nGreat. Now let's break down your monthly expenses. Just enter the amount for each.")
expenses['rent_or_emi'] = get_user_input("- Rent or Home Loan EMI:")
expenses['utilities'] = get_user_input("- Electricity, Water, Gas:")
expenses['internet_and_phone'] = get_user_input("- Internet & Phone Bills:")
expenses['groceries'] = get_user_input("- Groceries:")
expenses['eating_out'] = get_user_input("- Eating Out/Ordering In:")
expenses['transport'] = get_user_input("- Fuel/Public Transport:")
expenses['shopping'] = get_user_input("- Shopping (Clothes, etc.):")
expenses['entertainment'] = get_user_input("- Entertainment & Subscriptions:")
user_data['expenses_structured'] = expenses

# Gather goals, investments, and debts
user_data['financial_goals'] = get_user_input("\nWhat are your major financial goals (e.g., vacation, car, retirement)?")
user_data['current_investments'] = get_user_input("\nBriefly, what investments do you currently have (e.g., Mutual Funds, PPF, Stocks)?")
user_data['outstanding_debts'] = get_user_input("\nBriefly, what outstanding debts do you have (e.g., Credit Card, Personal Loan)?")

print("\nThank you for the information. Generating your personalized financial health report...")

# --- Phase 2: RAG-Powered Analysis ---

# Load the saved FAISS index
vectorstore = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) # Retrieve top 2 chunks 

# Use the smart formatting function
formatted_user_data = format_user_data_for_llm(user_data)

# The query used to find relevant information from your financial guide
analysis_query = "Provide a comprehensive financial health check-up, including analysis of cash flow, savings rate, and debt. Offer educational strategies on budgeting, debt management, and investing based on this user's data."

# Retrieve the context from your documents
retrieved_docs = retriever.invoke(analysis_query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Define the final prompt template
final_prompt_template = """
{instructions}

---
**CONTEXT FROM FINANCIAL GUIDE:**
{context}
---
**USER'S FINANCIAL DATA:**
{user_data}
---
**FINANCIAL HEALTH REPORT:**
"""
final_prompt = ChatPromptTemplate.from_template(final_prompt_template)

# --- Build and Invoke the RAG Chain ---
analysis_chain = (
    final_prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain with all necessary components
final_report = analysis_chain.invoke({
    "instructions": FIN_FRIEND_PROMPT_INSTRUCTIONS,
    "context": retrieved_context,
    "user_data": formatted_user_data
})


print("\n\n--- Your Financial Health Report ---\n")
print(final_report)

Hello! I'm Fin-Friend... Let's get started!

